In [ ]:
import csv
import cv2
import numpy as np
import random

lines = []
datapath = '../mydata/'
with open(datapath+'driving_log.csv') as csvfile:
    reader = csv.reader(csvfile)
    for line in reader:
        lines.append(line)
        
lines = lines[1:]        

from sklearn.model_selection import train_test_split
train_samples, validation_samples = train_test_split(lines, test_size=0.2)

import sklearn

def process_image(path):  
    return cv2.cvtColor(cv2.imread(path), cv2.COLOR_BGR2RGB)     

def generator(samples, batch_size=32):
    num_samples = len(samples)
    while 1:
        sklearn.utils.shuffle(samples)
        for offset in range(0, num_samples, batch_size):
            batch_samples = samples[offset:offset+batch_size]
            
            images=[]
            angles=[]
            for batch_sample in batch_samples:
                steering_center = float(batch_sample[3])
    
                correction = 0.2
                
                img_center = process_image(datapath+batch_sample[0])               

                original_images = [img_center]  
                original_steering = [steering_center] 
                
                if random.random() < 0.5:
                    img_left = process_image(datapath+batch_sample[1].strip())
                    steering_left = steering_center+correction
                    original_images.append(img_left)
                    original_steering.append(steering_left)
                else:
                    img_right = process_image(datapath+batch_sample[2].strip())
                    steering_right = steering_center-correction
                    original_images.append(img_right)
                    original_steering.append(steering_right)
                    
                for img,steer in zip(original_images,original_steering):

                    images.append(img)
                    angles.append(steer)

                    images.append(np.fliplr(img))
                    angles.append(-steer)   
            
            X_train = np.array(images)
            y_train = np.array(angles)
            yield sklearn.utils.shuffle(X_train, y_train)
            
train_generator = generator(train_samples, batch_size=32)
validation_generator = generator(validation_samples, batch_size=32)

ch, row, col = 3, 80, 320

from keras.models import Sequential
from keras.layers import Flatten, Dense, Lambda
from keras.layers import Convolution2D, MaxPooling2D, Cropping2D, Dropout

model = Sequential()
model.add(Lambda(lambda x: x/255.0 - 0.5, input_shape=(160,320,3)))
model.add(Cropping2D(cropping=((50,20),(0,0))))

model.add(Convolution2D(24,5,5,subsample=(2,2),activation="relu"))
model.add(Convolution2D(36,5,5,subsample=(2,2),activation="relu"))
model.add(Convolution2D(48,5,5,subsample=(2,2),activation="relu"))
model.add(Convolution2D(64,3,3,activation="relu"))
model.add(Convolution2D(64,3,3,activation="relu"))


model.add(Flatten())
model.add(Dense(1164, activation='linear'))
model.add(Dropout(.5))
model.add(Dense(100, activation='linear'))
model.add(Dropout(.5))
model.add(Dense(50, activation='linear'))
model.add(Dropout(.5))
model.add(Dense(10, activation='linear'))
model.add(Dense(1, activation='linear'))

model.compile(loss='mse', optimizer='adam')

history_object = model.fit_generator(train_generator, \
                                     samples_per_epoch=len(train_samples), \
                                     validation_data=validation_generator, \
                                     nb_val_samples=len(validation_samples), verbose=2, nb_epoch=10)
model.save('model.h5')

print(history_object.history.keys())

import matplotlib.pyplot as plt
%matplotlib inline 
plt.plot(history_object.history['loss'])
plt.plot(history_object.history['val_loss'])
plt.title('model mean squared error loss')
plt.ylabel('mean squared error loss')
plt.xlabel('epoch')
plt.legend(['training set', 'validation set'], loc='upper right')
plt.show()
